# Get Recommendations for Games

Ограничение: 4 запроса в секунду. Намного лучше, чем в Jikan

Доп.ограничение: не более 8 одновременно открытых сессий. Иначе - ошибка.

Twitch-токен нужно обновлять раз в n секунд. Пока держи это в голове.

In [18]:
import time
import asyncio
import aiohttp
import requests
import threading
import pandas as pd

In [ ]:
url = 'https://id.twitch.tv/oauth2/token?client_id=x0rncah74zueqqov31fhkwuzlqv27a&client_secret=nyoj0usmdecwegm7e14c40s9kbq1hw&grant_type=client_credentials'

In [46]:
response = requests.request('POST', url=url)

In [47]:
access_token = response.text

In [48]:
access_token

'{"access_token":"lhty0tvjq3el3s4rufgiw5vufmjk31","expires_in":4873986,"token_type":"bearer"}\n'

Все операции выше уже были проделаны. Токен был получен, его нужно будет заменить через 63 дня.

In [13]:
text_cols = ['storyline', 'summary']

In [14]:
cols = ['id', 'name',  'age_ratings', 'slug', 'aggregated_rating', 'aggregated_rating_count', 'category', 'collection', 
        'alternative_names', 'external_games', 'first_release_date', 'game_engines', 'game_modes', 'genres', 
        'involved_companies', 'keywords', 'platforms', 'rating', 'player_perspectives', 'dlcs', 'franchise',
        'rating_count', 'release_dates', 'similar_games', 'tags', 'parent_game', 'status',
        'total_rating', 'total_rating_count', 'updated_at', 'websites']

In [15]:
client_id = 'x0rncah74zueqqov31fhkwuzlqv27a'
access_token = 'Bearer lhty0tvjq3el3s4rufgiw5vufmjk31'

In [20]:
async def create_games_dataset(floor, ceiling, endpoint='games'):
    # нужно для того, чтобы авторизироваться
    headers = {'Client-ID': client_id, 
               'Authorization': access_token}
    async with aiohttp.ClientSession() as session:
        all_games = pd.DataFrame(columns=cols)
        for i in range(floor, ceiling):
            # endpoint и id можно изменять для того, чтобы получить разные наборы
            body = 'fields *; where id = ' + str(i) + ';'
            async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                game = await resp.json()
                try: # если вышла такая ошибка, значит слишком много запросов в одну секунду
                    # на тот случай, если id нет в базе данных
                    if len(game) > 0:
                        all_games.loc[len(all_games.index)] = game[0]
                except KeyError: # поэтому, ждем 1 секунду и делаем новый запрос
                    time.sleep(3)
                    async with session.post('https://api.igdb.com/v4/' + endpoint + '/', headers=headers, data=body) as resp:
                        game = await resp.json()
                        if len(game) > 0:
                            all_games.loc[len(all_games.index)] = game[0]
        return all_games

In [21]:
start_time = time.time()
all_fields_one = await create_games_dataset(1, 200)
print("--- %s seconds ---" % (time.time() - start_time))

<ClientResponse(https://api.igdb.com/v4/games/) [200 OK]>
<CIMultiDictProxy('Date': 'Mon, 04 Oct 2021 12:30:08 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '2715', 'Connection': 'keep-alive', 'x-amzn-RequestId': 'b8705fad-8412-441b-a370-29f979e45ed9', 'Content-Encoding': 'gzip', 'x-amzn-Remapped-Content-Length': '2715', 'x-amz-apigw-id': 'GrsIhGgRPHcFSUw=', 'x-amzn-Remapped-Server': 'Javalin', 'X-Count': '1', 'x-amzn-Remapped-Date': 'Mon, 04 Oct 2021 12:30:07 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 3bf14a720d62e0d1295d99086d103efb.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'ARN54-C1', 'X-Amz-Cf-Id': 'yi3gub4XpgPytM4cxuTrGvYr1UK7hr_vBGDJnd9DhOJa34je5qUX9A==', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'Server': 'cloudflare', 'CF-RAY': '698e6

<ClientResponse(https://api.igdb.com/v4/games/) [200 OK]>
<CIMultiDictProxy('Date': 'Mon, 04 Oct 2021 12:30:15 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '1336', 'Connection': 'keep-alive', 'x-amzn-RequestId': 'a18fa4c9-9384-45e7-ab1d-1a5ee798010c', 'Content-Encoding': 'gzip', 'x-amzn-Remapped-Content-Length': '1336', 'x-amz-apigw-id': 'GrsJsESdPHcFkag=', 'x-amzn-Remapped-Server': 'Javalin', 'X-Count': '1', 'x-amzn-Remapped-Date': 'Mon, 04 Oct 2021 12:30:15 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 0edd9f6bc061f9d64e77285b1cac290d.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'ARN54-C1', 'X-Amz-Cf-Id': 'us-Wyz0ourRYgrFDsPB02zEq1FOmdVyYsiRRT8xyTfipT0j3c3-eZA==', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'Server': 'cloudflare', 'CF-RAY': '698e6

<ClientResponse(https://api.igdb.com/v4/games/) [200 OK]>
<CIMultiDictProxy('Date': 'Mon, 04 Oct 2021 12:30:22 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '1278', 'Connection': 'keep-alive', 'x-amzn-RequestId': 'd9cea5f8-3fc9-4a8c-87f6-e6df48e69c83', 'Content-Encoding': 'gzip', 'x-amzn-Remapped-Content-Length': '1278', 'x-amz-apigw-id': 'GrsK2EvAPHcF30A=', 'x-amzn-Remapped-Server': 'Javalin', 'X-Count': '1', 'x-amzn-Remapped-Date': 'Mon, 04 Oct 2021 12:30:22 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 0dbf67e262a6295e9e8f6570f9aae7e0.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'ARN54-C1', 'X-Amz-Cf-Id': 'B_VspOWoDQ3GJ0UkpBqVsDKFWf9tply3-Z-_mHkHcHh7V9MES_5ezA==', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'Server': 'cloudflare', 'CF-RAY': '698e6

<ClientResponse(https://api.igdb.com/v4/games/) [200 OK]>
<CIMultiDictProxy('Date': 'Mon, 04 Oct 2021 12:30:29 GMT', 'Content-Type': 'application/json;charset=utf-8', 'Content-Length': '2679', 'Connection': 'keep-alive', 'x-amzn-RequestId': 'fcee3a9c-1a88-4f48-8d3a-87939e2dc56b', 'Content-Encoding': 'gzip', 'x-amzn-Remapped-Content-Length': '2679', 'x-amz-apigw-id': 'GrsL5F9ZvHcFuBQ=', 'x-amzn-Remapped-Server': 'Javalin', 'X-Count': '1', 'x-amzn-Remapped-Date': 'Mon, 04 Oct 2021 12:30:29 GMT', 'X-Cache': 'Miss from cloudfront', 'Via': '1.1 432d52d55ad517cddd9081b248b2f116.cloudfront.net (CloudFront)', 'X-Amz-Cf-Pop': 'ARN54-C1', 'X-Amz-Cf-Id': 'vTO8MOPcTjiCCXNHzWjYwjm_YdTtF3FMt6igHP23v8rQuj5gJTYGZA==', 'CF-Cache-Status': 'DYNAMIC', 'Expect-CT': 'max-age=604800, report-uri="https://report-uri.cloudflare.com/cdn-cgi/beacon/expect-ct"', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains; preload', 'X-Content-Type-Options': 'nosniff', 'Server': 'cloudflare', 'CF-RAY': '698e6

CancelledError: 